In [2]:
import tensorflow as tf

2024-02-19 20:40:31.178717: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-19 20:40:31.201048: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 20:40:31.201073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 20:40:31.201811: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 20:40:31.205666: I tensorflow/core/platform/cpu_feature_guar

In [1]:
def gen_model_dnn(layers, model_in_path):
    if len(layers) == 3:
        inputs = tf.keras.layers.Input(shape=(layers[0],))
        out = tf.keras.layers.Dense(layers[1], activation = 'relu')(inputs)
        out = tf.keras.layers.Dense(layers[2])(out)

        model = tf.keras.Model(inputs, out)

    elif len(layers) == 4:
        inputs = tf.keras.layers.Input(shape=(layers[0],))
        out = tf.keras.layers.Dense(layers[1], activation = 'relu')(inputs)
        out = tf.keras.layers.Dense(layers[2], activation = 'relu')(out)
        out = tf.keras.layers.Dense(layers[3])(out)

        model = tf.keras.Model(inputs, out)
    else:
        print ("Layers not Support")
        return None
    
    model.load_weights(model_in_path)
    model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
    return model

# @ TODO: Hardcoded
def gen_model_cnn(layers, model_in_path):
    kernal_size = layers[-1]

    # Define the LeNet model in TensorFlow
    inputs = tf.keras.layers.Input(shape=(layers[0],layers[0],1))
    out = tf.keras.layers.Conv2D(layers[1],kernal_size, use_bias = True)(inputs)
    out = tf.keras.layers.ReLU()(out)
    out = tf.keras.layers.AveragePooling2D()(out)
    out = tf.keras.layers.Conv2D(layers[2],kernal_size, use_bias = True)(out)
    out = tf.keras.layers.ReLU()(out)
    out = tf.keras.layers.AveragePooling2D()(out)
    out = tf.keras.layers.Flatten()(out)
    out = tf.keras.layers.Dense(layers[3])(out)

    if len(layers) >= 6:
        out = tf.keras.layers.ReLU()(out)
        out = tf.keras.layers.Dense(layers[4])(out)

    if len(layers) >= 7:
        out = tf.keras.layers.ReLU()(out)
        out = tf.keras.layers.Dense(layers[5])(out)

    model = tf.keras.Model(inputs, out)

    model.load_weights(model_in_path)

    # Compile the model
    model.compile(optimizer='adam',
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])
    return model

In [3]:
def get_model_path(layers):
    if layers[0] > 30:
        arch_folder = "./input" + (len(layers)-1) * "-dense" + "/"
    else:
        arch_folder = "./input-conv2d-conv2d" + (len(layers) - 4) * "-dense" + "/"

    return arch_folder + '_'.join([str(x) for x in layers]) + '.h5'


In [4]:
def get_model_info(layers):
    if layers[0] == 784 or layers[0] == 28:
        tests = test_images_tf
    else:
        tests = test_images_tf_downsampled

    if layers[0] > 30:
        model_in_path = get_model_path(layers)
        model = gen_model_dnn(layers, model_in_path)
        model.summary()
    else:
        model_in_path = get_model_path(layers)
        model = gen_model_cnn(layers, model_in_path)
        model.summary()      

    # Evaluate the model
    test_loss, test_acc = model.evaluate(tests, test_labels, verbose=2)
    print('\nTest accuracy:', test_acc)


## DNN

In [23]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Resize for 14 * 14 images
train_images_tf_reshaped = tf.reshape(train_images_tf, [-1, 28, 28, 1])  # Reshape to [num_samples, height, width, channels]
test_images_tf_reshaped = tf.reshape(test_images_tf, [-1, 28, 28, 1])

# Downsample images
train_images_tf_downsampled = tf.image.resize(train_images_tf_reshaped, [14, 14], method='bilinear')
test_images_tf_downsampled = tf.image.resize(test_images_tf_reshaped, [14, 14], method='bilinear')

# Flatten the images back to [num_samples, 14*14]
train_images_tf_downsampled = tf.reshape(train_images_tf_downsampled, [-1, 14*14])
test_images_tf_downsampled = tf.reshape(test_images_tf_downsampled, [-1, 14*14])

In [20]:
layers = [196,25,10]
get_model_info(layers)

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense_13 (Dense)            (None, 25)                4925      
                                                                 
 dense_14 (Dense)            (None, 10)                260       
                                                                 
Total params: 5185 (20.25 KB)
Trainable params: 5185 (20.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 - 0s - loss: 0.8117 - accuracy: 0.9541 - 324ms/epoch - 1ms/step

Test accuracy: 0.9541000127792358


In [19]:
layers = [196,24,14,10]
get_model_info(layers)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 196)]             0         
                                                                 
 dense_10 (Dense)            (None, 24)                4728      
                                                                 
 dense_11 (Dense)            (None, 14)                350       
                                                                 
 dense_12 (Dense)            (None, 10)                150       
                                                                 
Total params: 5228 (20.42 KB)
Trainable params: 5228 (20.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 - 0s - loss: 0.5499 - accuracy: 0.9556 - 416ms/epoch - 1ms/step

Test accuracy: 0.9556000232696533


In [21]:
layers = [784,56,10]
get_model_info(layers)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_15 (Dense)            (None, 56)                43960     
                                                                 
 dense_16 (Dense)            (None, 10)                570       
                                                                 
Total params: 44530 (173.95 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
313/313 - 0s - loss: 0.5807 - accuracy: 0.9740 - 258ms/epoch - 823us/step

Test accuracy: 0.9739999771118164


## CNN

In [8]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images_tf = train_images / 255.0
test_images_tf = test_images / 255.0
train_images_tf = train_images_tf.reshape(train_images.shape[0], 28, 28, 1)
test_images_tf = test_images_tf.reshape(test_images.shape[0], 28, 28, 1)

train_images_tf_downsampled = tf.image.resize(train_images_tf, [14, 14]).numpy()
test_images_tf_downsampled = tf.image.resize(test_images_tf, [14, 14]).numpy()

In [6]:
layers = [28,6,16,10,5]
get_model_info(layers)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 24, 24, 6)         156       
                                                                 
 re_lu (ReLU)                (None, 24, 24, 6)         0         
                                                                 
 average_pooling2d (Average  (None, 12, 12, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 re_lu_1 (ReLU)              (None, 8, 8, 16)          0         
                                                             

2024-02-19 20:40:52.872979: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-02-19 20:40:52.912375: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 - 1s - loss: 0.0417 - accuracy: 0.9866 - 666ms/epoch - 2ms/step

Test accuracy: 0.9865999817848206


In [9]:
layers = [14,5,11,80,10,3]
get_model_info(layers)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 14, 14, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 5)         50        
                                                                 
 re_lu_2 (ReLU)              (None, 12, 12, 5)         0         
                                                                 
 average_pooling2d_2 (Avera  (None, 6, 6, 5)           0         
 gePooling2D)                                                    
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 4, 11)          506       
                                                                 
 re_lu_3 (ReLU)              (None, 4, 4, 11)          0         
                                                           

2024-02-19 20:41:44.150879: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


313/313 - 0s - loss: 0.0971 - accuracy: 0.9707 - 403ms/epoch - 1ms/step

Test accuracy: 0.9707000255584717


In [10]:
layers = [28,6,16,120,84,10,5]
get_model_info(layers)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 6)         156       
                                                                 
 re_lu_5 (ReLU)              (None, 24, 24, 6)         0         
                                                                 
 average_pooling2d_4 (Avera  (None, 12, 12, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 re_lu_6 (ReLU)              (None, 8, 8, 16)          0         
                                                           